In [1]:
import torch
from torch import nn
from dataloader import dataset, collate_fn
import pickle
from torch.utils import data

In [2]:
data_ = []
file_path = './generated_dataset.txt'
with open(file_path, 'r') as f:
    for i in f:
        j = i.strip('\n').split('\t')
        data_.append(j[-1].lower())

tweet_pairs, distance_vectors = [], []
with open('./tweet_pairs.pkl', 'rb') as f:
    tweet_pairs = pickle.load(f)
with open('./distance_vectors.pkl', 'rb') as f:
    distance_vectors = pickle.load(f)
    
tweet_pair_data = [[data_[i[0]], data_[i[1]]] for i in tweet_pairs]
distance_vector_data = [[distance_vectors[i[0]],distance_vectors[i[1]]] for i in tweet_pairs]

dataset_ = dataset(tweet_pair_data, distance_vector_data)
loader = data.DataLoader(dataset_, batch_size=32, collate_fn=collate_fn, shuffle=True)

In [9]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        with open("./vocab.pkl", "rb") as f:
            self.vocab = pickle.load(f)
        pre_trained_emb = torch.Tensor(self.vocab.vectors)
        self.word_embedding = nn.Embedding.from_pretrained(pre_trained_emb)
        self.distance_embedding = nn.Embedding(64, 14)
    
    def forward(self, tweet1, tweet2, dist1, dist2):
        tweet1_embedding = self.word_embedding(tweet1.long())
        tweet2_embedding = self.word_embedding(tweet2.long())
        try:
            dist1_embedding = self.distance_embedding(dist1.long())
            dist2_embedding = self.distance_embedding(dist2.long())
            return tweet1_embedding.shape, tweet2_embedding.shape, dist1_embedding.shape, dist2_embedding.shape            
            print("ERROR")
            print(dist1, dist2)


In [10]:
model = Model()

In [11]:
for a, b, c, d in loader:
    model(a, b, c, d)
    break

ERROR
tensor([[ -9.,  -8.,  -7.,  ..., -28., -28., -28.],
        [ -2.,  -1.,   0.,  ..., -28., -28., -28.],
        [-27., -26., -25.,  ..., -28., -28., -28.],
        ...,
        [ -9.,  -8.,  -7.,  ..., -28., -28., -28.],
        [ -4.,  -3.,  -2.,  ..., -28., -28., -28.],
        [ -5.,  -4.,  -3.,  ..., -28., -28., -28.]]) tensor([[ -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,
           6.,   7.,   8., -28., -28., -28., -28., -28., -28., -28., -28., -28.,
         -28., -28., -28.],
        [ -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4., -28.,
         -28., -28., -28., -28., -28., -28., -28., -28., -28., -28., -28., -28.,
         -28., -28., -28.],
        [ -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,
         -28., -28., -28., -28., -28., -28., -28., -28., -28., -28., -28., -28.,
         -28., -28., -28.],
        [ -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,
          10.,  1